<a href="https://colab.research.google.com/github/bxck75/piss-ant-pix2pix/blob/master/modeltransferv2_pailkids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# remove defaults
!rm -r sample_data
# Clone the repo
!git clone https://github.com/bxck75/Python_Helpers.git

# Change dir
%cd /content/Python_Helpers
# install
!python setup.py install

from IPython.display import clear_output
import main
import os
import sys
import IPython
import Helpers
P=Helpers.core.Core()
hlp=P.H
clear_output()

In [0]:
# get repo
%cd /content/
hlp.repolist= hlp.repo_collection
repos_sorted = hlp.repo_collection.repos_sorted
A1=['bxck75/piss-ant-pix2pix']
hlp.Me(['inst_reps',A1,'/content/installed_repos',False,True])
clear_output()

In [0]:
# get images
%cd /content/
# images_set_name='insect'
# images_set_code='1fbEUcBIvLwjlVFrUiW2g8KtRTvFSsW9D'
# images_model_code='1eFR5ZHfp5PzPEnFv_v3xxE9T0tKPJTS4'

# https://drive.google.com/open?id=1PUEpLgRfKKDvaMAVtKBVr-Z0ThQvOGWv # images
# https://drive.google.com/open?id=1vLmCOaActAAZ0bHdrv0YgP-rCnYHX0O9

# images_set_name='apes'
# images_set_code='1PUEpLgRfKKDvaMAVtKBVr-Z0ThQvOGWv'
# images_model_code='1vLmCOaActAAZ0bHdrv0YgP-rCnYHX0O9'

# https://drive.google.com/open?id=1nb8yr9TIJaiB1GFN9S4mD5GfTNvEQpW-
images_set_name='garbagepailkids'
images_set_code='1nb8yr9TIJaiB1GFN9S4mD5GfTNvEQpW-'
images_model_code='1tp8Gm1fMuSQoLEEXQ_C1Dlw1N4wwFxKb'

Helpers.GdriveD.GdriveD(images_set_code, '/content/'+images_set_name+'.zip')
!unzip /content/{images_set_name}.zip
clear_output()

In [0]:
# make dataset
%cd /content/installed_repos/piss-ant-pix2pix
!bash image_folder_make_set_train_colab.sh /content/{images_set_name} 1 prep
clear_output()

In [0]:
''' 
    startup loop  1x
    # 1 get metric from gdrive
    2 train 5 epochs
    3 dump metrics to gdrive with the same file id 
'''
for i in range(1):
    # fetch metrics
    %cd /content/
    # Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
    # !unzip /content/{images_set_name}_model_in.zip -d metrics/
    # !rm -r /content/metrics/options.json
    # train epochs
    %cd /content/installed_repos/piss-ant-pix2pix
    !python pix2pix.py --output_dir /content/metrics --progress_freq 100 --save_freq 150 --summary_freq 100 --display_freq 100 --max_epochs 1 --mode train --input_dir /content/{images_set_name}images/_combined/train --which_direction 'BtoA'
    # dump metrics
    %cd /content/
    Helpers.Core()
    obj=Helpers.core.Core()
    folder_of_model='metrics'
    result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
    zip_hash=result.split('(id) ')[1]
    print(zip_hash)
    # delete old crap
    !rm -r /content/metrics
    !rm -r /content/{images_set_name}model.zip

start values
progress  epoch 1  step 10  image/sec 0.7  remaining 122m
dloss_GAN 0.11989411
gloss_GAN 0.060013566
gloss_L1 0.043864187

In [0]:
''' 
    loop  10x
    1 get metric from gdrive
    2 train 5 epochs
    3 dump metrics to gdrive with the same file id 
'''
for i in range(10):
    # fetch metrics
    %cd /content/
    Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
    !unzip /content/{images_set_name}_model_in.zip -d metrics/
    !rm -r /content/metrics/options.json
    # train epochs
    %cd /content/installed_repos/piss-ant-pix2pix
    !python pix2pix.py --checkpoint /content/metrics --output_dir /content/metrics --progress_freq 10 --save_freq 50 --summary_freq 50 --display_freq 200 --max_epochs 5 --mode train --input_dir /content/{images_set_name}images/_combined/train --which_direction 'BtoA'
    # dump metrics
    %cd /content/
    # delete unwanted
    !rm -r /content/metrics/images
    !rm -r /content/metrics/events.out.tfevents*
    !rm -r /content/metrics/index.html
    Helpers.Core()
    obj=Helpers.core.Core()
    folder_of_model='metrics'
    result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
    zip_hash=result.split('(id) ')[1]
    print(zip_hash)
    # delete old crap
    !rm -r /content/metrics
    !rm -r /content/{images_set_name}model.zip

In [0]:
# # rewrite checkpointfile
# !echo 'model_checkpoint_path: "/content/metrics/model-21880"' > /content/metrics/checkpoint
# !echo 'all_model_checkpoint_paths: "/content/metrics/model-21880"' >> /content/metrics/checkpoint

In [0]:
# download/unzip metrics 
# Get pretrained metrics
%cd /content/
Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
!unzip /content/{images_set_name}_model_in.zip -d metrics/

In [0]:
# remove old crap before rezipping
!rm -r /content/metrics/images
!rm -r /content/metrics/events.out.tfevents*
!rm -r /content/metrics/index.html

In [0]:
# rezip
%cd /content/
Helpers.Core()
obj=Helpers.core.Core()
folder_of_model='metrics'
print(obj.Gdrive_root+'/models')
result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
# print(result)
zip_hash=result.split('(id) ')[1]
print(zip_hash)